In [2]:
import jsonpickle

import json
import pprint

def json_pretty_print(parsed):
    """
    Pretty print a json object.
    """
    if isinstance(parsed, str):
        parsed = json.loads(parsed)
    #ret = pprint.pformat(parsed)
    ret = json.dumps(parsed, indent=4, sort_keys=True)
    return ret

import jsonpickle.ext.pandas as jsonpickle_pandas
jsonpickle_pandas.register_handlers()

In [3]:
import inspect

def round_trip(obj):
    print("obj=\n", obj)
    frozen = jsonpickle.encode(obj)
    #
    print("frozen=")
    print(json_pretty_print(frozen))
    #
    obj2 = jsonpickle.decode(frozen)
    print("obj2=\n", obj2)
    print("class=%s" % type(obj))
    if str(type(obj)).startswith("<class '"):
        pass
    else:
        if isinstance(obj, pd.DataFrame):
            assert obj.equals(obj2)
        else:
            assert obj == obj2
    return obj2

In [4]:
import pandas as pd

data = {
    'Product': ['Desktop Computer', 'Tablet', 'iPhone', 'Laptop'],
    'Price': [700, 250, 800, 1200]
}

df = pd.DataFrame(data, columns=['Product', 'Price'])
df.index.name = "hello"
print(df)

                Product  Price
hello                         
0      Desktop Computer    700
1                Tablet    250
2                iPhone    800
3                Laptop   1200


In [18]:
round_trip(3)

obj=
 3
frozen=
3
obj2=
 3
class=<class 'int'>


3

In [38]:
class Playback:
    
    #def __init__(self, file_name, mode, *args, **kwargs):
    # self.args = args
    # self.kwargs = kwargs
    def __init__(self, file_name, mode, func_name, a, b):
        self.a = a
        self.b = b
        
    def start(self):
        self.a_json = jsonpickle.encode(self.a)
        self.b_json = jsonpickle.encode(self.b)
    
    def end(self, ret):
        self.ret_json = jsonpickle.encode(ret)
        output = []
        output.append("# Initialize values for unit test.")
        output.append("a = %s" % jsonpickle.decode(self.a_json))
        output.append("b = %s" % jsonpickle.decode(self.b_json))
        output.append("# Apply values.")
        output.append("act = F(a, b)")
        output.append("exp = %s" % jsonpickle.decode(self.ret_json))
        #output.append("self.assertEqual(act, exp)")
        output.append("assert act == exp")
        output = "\n".join(output)
        print("output=", output)
        
        
# def F(a: int, b: int):
#     c = {}
#     c["pavel"] = a + b
#     return c
        

def F(a: int, b: int):
    playback = Playback("", "", "F", a, b)
    playback.start()
    c = {}
    c["pavel"] = a + b
    playback.end(c)
    return c
    
    
res = F(3, 4)
print(res)

output= # Initialize values for unit test.
a = 3
b = 4
# Apply values.
act = F(a, b)
exp = {'pavel': 7}
assert act == exp
{'pavel': 7}


In [39]:
# Initialize values for unit test.
a = 3
b = 4
# Apply values.
act = F(a, b)
exp = {'pavel': 7}
assert act == exp

output= # Initialize values for unit test.
a = 3
b = 4
# Apply values.
act = F(a, b)
exp = {'pavel': 7}
assert act == exp


In [5]:
df2 = round_trip(df)

obj=
                 Product  Price
hello                         
0      Desktop Computer    700
1                Tablet    250
2                iPhone    800
3                Laptop   1200
frozen=
{
    "meta": {
        "dtypes": {
            "Price": "int64",
            "Product": "object"
        },
        "index": "{\"py/object\": \"pandas.core.indexes.range.RangeIndex\", \"values\": \"[0, 1, 2, 3]\", \"txt\": true, \"meta\": {\"dtype\": \"int64\", \"name\": \"hello\"}}"
    },
    "py/object": "pandas.core.frame.DataFrame",
    "txt": true,
    "values": "Product,Price\nDesktop Computer,700\nTablet,250\niPhone,800\nLaptop,1200\n"
}
obj2=
                 Product  Price
hello                         
0      Desktop Computer    700
1                Tablet    250
2                iPhone    800
3                Laptop   1200
class=<class 'pandas.core.frame.DataFrame'>


In [ ]:
class Thing:
    def __init__(self, name):
        self.name = name
        
obj = Thing('Awesome')

round_trip(obj)

In [ ]:
def test(a: int, b: int):
    print(round_trip(a))

    
test("strunz", 6)
test(4, 6)
test(["hello"], 6)

In [ ]:
df.index.dtype#

In [ ]:
df.dtypes

In [13]:
#import io
#import io.StringIO
#from io import StringIO

#output = StringIO.StringIO()

orient = "columns"
#orient = "split"
#orient = "records"
#orient = "table"
df_as_str = df.to_json(orient=orient)

# split
# records
# index
# values
# table
# columns (the default format)

python_code = []
target_var = "df_as_str"
python_code.append("%s = %s" % (target_var, df_as_str))
python_code.append("%s.index.name = '%s'" % (target_var, df.index.name))
python_code = "\n".join(python_code)
print(python_code)

exec(python_code)

df_as_str = {"Product":{"0":"Desktop Computer","1":"Tablet","2":"iPhone","3":"Laptop"},"Price":{"0":700,"1":250,"2":800,"3":1200}}
df_as_str.index.name = 'hello'


AttributeError: 'dict' object has no attribute 'index'

In [ ]:
airr = eval(df_as_str)
df2 = pd.DataFrame.from_dict(arr, orient="columns")
df2.index.name